In [ ]:
import pandas as pd
from matplotlib import pyplot as plt

: 

In [105]:
df = pd.read_csv('./data/processed/cars.csv')
print(df.shape)
df.head()

(8128, 13)


,name,year,selling_price,km_driven,fuel,seller_type,transmission,owner,mileage,engine,max_power,torque,seats
0,Maruti Swift Dzire VDI,2014,450000,145500,Diesel,Individual,Manual,First Owner,23.40,1248 CC,74 bhp,190Nm@ 2000rpm,5.0
1,Skoda Rapid 1.5 TDI Ambition,2014,370000,120000,Diesel,Individual,Manual,Second Owner,21.14,1498 CC,103.52 bhp,250Nm@ 1500-2500rpm,5.0
2,Honda City 2017-2020 EXi,2006,158000,140000,Petrol,Individual,Manual,Third Owner,17.70,1497 CC,78 bhp,"12.7@ 2,700(kgm@ rpm)",5.0
3,Hyundai i20 Sportz Diesel,2010,225000,127000,Diesel,Individual,Manual,First Owner,23.00,1396 CC,90 bhp,22.4 kgm at 1750-2750rpm,5.0
4,Maruti Swift VXI BSIII,2007,130000,120000,Petrol,Individual,Manual,First Owner,16.10,1298 CC,88.2 bhp,"11.5@ 4,500(kgm@ rpm)",5.0


In [106]:
df = df[df.isna().sum(axis=1) == 0]
df = df[df.seller_type != 'Trustmark Dealer']
df = df[df.selling_price < 0.2 * 1e7]
df.shape

(7301, 13)

In [107]:
df = df.assign(
    year=df.year.max() - df.year,
)

df.head(3)

,name,year,selling_price,km_driven,fuel,seller_type,transmission,owner,mileage,engine,max_power,torque,seats
0,Maruti Swift Dzire VDI,6,450000,145500,Diesel,Individual,Manual,First Owner,23.40,1248 CC,74 bhp,190Nm@ 2000rpm,5.0
1,Skoda Rapid 1.5 TDI Ambition,6,370000,120000,Diesel,Individual,Manual,Second Owner,21.14,1498 CC,103.52 bhp,250Nm@ 1500-2500rpm,5.0
2,Honda City 2017-2020 EXi,14,158000,140000,Petrol,Individual,Manual,Third Owner,17.70,1497 CC,78 bhp,"12.7@ 2,700(kgm@ rpm)",5.0


In [108]:
from math import log2

df = df.assign(
    year_0_5=((df.year <= 5)).astype(int),
    year_5_9=((df.year > 5) & (df.year <= 10)).astype(int),
    year_9_=((df.year > 10)).astype(int),
    selling_price=df.selling_price.apply(log2)
)

df.head(3)

,name,year,selling_price,km_driven,fuel,seller_type,transmission,owner,mileage,engine,max_power,torque,seats,year_0_5,year_5_9,year_9_
0,Maruti Swift Dzire VDI,6,18.779565,145500,Diesel,Individual,Manual,First Owner,23.40,1248 CC,74 bhp,190Nm@ 2000rpm,5.0,0,1,0
1,Skoda Rapid 1.5 TDI Ambition,6,18.497166,120000,Diesel,Individual,Manual,Second Owner,21.14,1498 CC,103.52 bhp,250Nm@ 1500-2500rpm,5.0,0,1,0
2,Honda City 2017-2020 EXi,14,17.269565,140000,Petrol,Individual,Manual,Third Owner,17.70,1497 CC,78 bhp,"12.7@ 2,700(kgm@ rpm)",5.0,0,0,1


# ml

In [109]:
from sklearn.linear_model import LinearRegression
from sklearn.neighbors import KNeighborsRegressor
from sklearn.model_selection import train_test_split
from sklearn.metrics import mean_absolute_percentage_error

In [110]:
cols = [
 # 'name',
 # 'year',
 'km_driven',
 'fuel',
 'seller_type',
 'transmission',
 # 'owner',
 'mileage',
 # 'engine',
 # 'max_power',
 # 'torque',
 'seats',
    
 'year_0_5',
 'year_5_9',
 'year_9_',
    
 'selling_price',
]

df = df[cols]
df.head()

,km_driven,fuel,seller_type,transmission,mileage,seats,year_0_5,year_5_9,year_9_,selling_price
0,145500,Diesel,Individual,Manual,23.40,5.0,0,1,0,18.779565
1,120000,Diesel,Individual,Manual,21.14,5.0,0,1,0,18.497166
2,140000,Petrol,Individual,Manual,17.70,5.0,0,0,1,17.269565
3,127000,Diesel,Individual,Manual,23.00,5.0,0,1,0,17.779565
4,120000,Petrol,Individual,Manual,16.10,5.0,0,0,1,16.988152


In [111]:
df = pd.get_dummies(df)
print(df.shape)
df.head()

(7301, 15)


,km_driven,mileage,seats,year_0_5,year_5_9,year_9_,selling_price,fuel_CNG,fuel_Diesel,fuel_LPG,fuel_Petrol,seller_type_Dealer,seller_type_Individual,transmission_Automatic,transmission_Manual
0,145500,23.40,5.0,0,1,0,18.779565,0,1,0,0,0,1,0,1
1,120000,21.14,5.0,0,1,0,18.497166,0,1,0,0,0,1,0,1
2,140000,17.70,5.0,0,0,1,17.269565,0,0,0,1,0,1,0,1
3,127000,23.00,5.0,0,1,0,17.779565,0,1,0,0,0,1,0,1
4,120000,16.10,5.0,0,0,1,16.988152,0,0,0,1,0,1,0,1


In [112]:
train, test = train_test_split(df, test_size=0.5)

target_col = 'selling_price'
train_x, train_y = train.drop(target_col, axis=1), train[target_col]
test_x, test_y = test.drop(target_col, axis=1), test[target_col]

In [ ]:
model = LinearRegression()
model.fit(train_x, train_y)

pred = model.predict(test_x)
mean_absolute_percentage_error(2**test_y, 2**pred)

: 